In [2]:
#Importing the libraries
import numpy as np
from tqdm import tqdm
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [3]:
os.listdir()

['README.md',
 '.git',
 'Text Generation.ipynb',
 'LICENSE',
 'scifi-stories-text-corpus.zip',
 '.ipynb_checkpoints',
 'internet_archive_scifi_v3.txt']

In [4]:
!unzip scifi-stories-text-corpus.zip

Archive:  scifi-stories-text-corpus.zip
  inflating: internet_archive_scifi_v3.txt  


In [4]:
!sudo chmod 777 internet_archive_scifi_v3.txt
!stat -c "%a %n" internet_archive_scifi_v3.txt

777 internet_archive_scifi_v3.txt


In [6]:
#Load the raw text and convert them to lowercase
filename = "internet_archive_scifi_v3.txt"
text_corpus = open(filename).read()
len_text = len(text_corpus)
text_corpus = text_corpus[0:len_text//100]
text_corpus = text_corpus.lower()

In [7]:
len(text_corpus)

1493263

In [8]:
#Create mapping from characters to integers
chars = sorted(list(set(text_corpus)))
chars_to_int = dict((char, integer) for integer, char in enumerate(chars))

In [9]:
#Print the unique characters present in the corpus
print(chars_to_int.keys())

dict_keys(['t', 'q', ' ', '?', 's', ';', 'p', 'j', 'i', 'w', 'l', 'f', "'", '(', ')', 'm', 'z', 'n', '"', ':', 'b', 'u', 'a', '#', 'y', 'e', 'k', '-', 'r', 'd', 'h', ',', '.', '!', 'v', 'c', 'x', 'g', 'o'])


In [10]:
#Basic information about the dataset
n_chars = len(text_corpus)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  1493263
Total Vocab:  39


In [11]:
#Prepare the dataset of input to output pairs encoded as integers
seq_length = 100
X_data = []
y_data = []
for i in tqdm(range(0, n_chars - seq_length, 1)):
    input_sequence = text_corpus[i:i + seq_length]
    output_sequence = text_corpus[i + seq_length]
    X_data.append([chars_to_int[char] for char in input_sequence])
    y_data.append(chars_to_int[output_sequence])
n_patterns = len(X_data)
print ("Total Patterns: ", n_patterns)

100%|██████████| 1493163/1493163 [00:19<00:00, 75437.67it/s] 

Total Patterns:  1493163


In [12]:
#Reshape X to be [samples, time steps, features]
X = np.reshape(X_data, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(y_data)

In [14]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [15]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

W0813 06:55:42.419023 140672944252672 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
  44928/1493163 [..............................] - ETA: 37:45 - loss: 3.0060